In [ ]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

spark = SparkSession.builder.config('spark,ui,port',3000).getOrCreate()

In [ ]:
path = "s3n://mybuck3t/sparkify_log_small_error.json"
logs3 = spark.read.json(path)

In [ ]:
logs3.head(2)

In [ ]:
logs3.where(logs3["_corrupt_record"].isNotNull()).collect()

In [ ]:
incorrect_records = SparkContext.accumulator(0,0)

In [ ]:
incorrect_records.value

In [ ]:
def add_incorrect_record():
    global incorrect_records
    incorrect_records += 1

In [ ]:
from pyspark.sql.functions import udf
correct_ts = udf(lambda x: 1 if x.isdigit() else add_incorrect_record())
#correct_ts = udf(lambda x: 1 if str(x).isdigit() else add_incorrect_record())

In [ ]:
logs3 = logs3.where(logs3["_corrupt_record"].isNull()).withColumn("ts_digit", correct_ts(logs3["ts"]))

In [ ]:
incorrect_records.value

In [ ]:
logs3.collect()